In [1]:
from sklearn.neighbors import KNeighborsRegressor as KNN
from sklearn.preprocessing import StandardScaler

from joblib import dump, load
import pandas as pd
import numpy as np

from psyke import Extractor, Clustering, Target
from psyke.extraction.hypercubic.strategy import AdaptiveStrategy
from psyke.extraction.hypercubic import Grid, FeatureRanker
from psyke.utils.logic import pretty_theory

In [2]:
def getTrainTest(data, testB):
    b = bartels[bartels.n==testB]
    t0, t1 = b.t0.values[0], b.t1.values[0]
    idx = (data.index >= t0) & (data.index < t1)
    return data[~idx], data[idx]

def getScaler(train, name):
    scaler = StandardScaler().fit(train)
    dump(scaler, f"scalers/scalerV{name}.joblib")
    normalization = {key: (m, s) for key, m, s in zip(train.columns, scaler.mean_, scaler.scale_)}
    return scaler, pd.DataFrame(scaler.transform(train), columns=train.columns), normalization

In [3]:
def gridex(model, train, test, normalization, s, m):
    ranked = FeatureRanker(train.columns).fit(model, train.iloc[:, :-1]).rankings()
    gridEx = Extractor.gridex(model, Grid(1, AdaptiveStrategy(ranked, [(0.6, 3), (0.75, 4)])),
                              threshold=5, min_examples=1, normalization=normalization)
    gridEx.extract(train)
    return gridEx.brute_predict(test), gridEx.n_rules, sum([p is None for p in gridEx.predict(test)])
    
def gridrex(model, train, test, normalization, s, m):
    ranked = FeatureRanker(train.columns).fit(model, train.iloc[:, :-1]).rankings()
    gridREx = Extractor.gridrex(model, Grid(1, AdaptiveStrategy(ranked, [(0.5, 3)])),
                                threshold=5, min_examples=1, normalization=normalization)
    gridREx.extract(train)
    return gridREx.brute_predict(test), gridREx.n_rules, sum([p is None for p in gridREx.predict(test)])

def cart(model, train, test, normalization, s, m):
    CART = Extractor.cart(model, max_depth=5, max_leaves=7, normalization=normalization)
    CART.extract(train)
    return CART.predict(test) * s + m, CART.n_rules, sum([p is None for p in CART.predict(test)])

def cosmik(model, train, test, normalization, s, m):
    COSMiK = Extractor.cosmik(model, max_components=10, k=100, patience=10, close_to_center=True,
                              output=Target.CONSTANT, normalization=normalization)
    COSMiK.extract(train)
    return COSMiK.brute_predict(test), COSMiK.n_rules, sum([p is None for p in COSMiK.predict(test)])

def creepy(model, train, test, normalization, s, m):
    CReEPy = Extractor.creepy(model, clustering=Clustering.cream, depth=5, error_threshold=5, gauss_components=10,
                              output=Target.REGRESSION, normalization=normalization)
    CReEPy.extract(train)
    return CReEPy.brute_predict(test), CReEPy.n_rules, sum([p is None for p in CReEPy.predict(test)])

In [4]:
bartels = pd.read_csv("data/bartels.csv", parse_dates = [1, 2])

TESTB = [i for i in range(2491, 2509)]

predicted = {'index': [], 'V': [], 'model': [], 'GridREx': [], 'CART': [], 'COSMiK': [], 'CReEPy': []}

rules = {'BR': [], 'GridREx': [], 'CART': [], 'COSMiK': [], 'CReEPy': []}

missed = {'BR': [], 'GridREx': [], 'CART': [], 'COSMiK': [], 'CReEPy': []}

for testB in TESTB:
    rules['BR'].append(testB)
    missed['BR'].append(testB)
    print(testB)

    data = pd.read_csv(f'data/halffuzzycoefs2B.csv', parse_dates=[0], index_col=0)
    train, test = getTrainTest(data, testB)

    predicted['index'] += list(test.index.values)
    predicted['V'] += list(test.V.values)

    scaler, scaledTrain, normalization = getScaler(train, f"test{testB}")
    scaledTest = pd.DataFrame(scaler.transform(test), columns=test.columns).iloc[:, :-1]
    m, s = normalization[test.columns[-1]]

    model = KNN(200, weights='distance', p=1).fit(scaledTrain.iloc[:, :-1], scaledTrain.iloc[:, -1])
    #dump(model, f"models/RF/{k}_{name}_{testB}.joblib")
    predicted['model'] += list(model.predict(scaledTest) * s + m)

    for name, fun in zip(['GridREx', 'CART', 'COSMiK', 'CReEPy'], [gridrex, cart, cosmik, creepy]):
        print(name)
        #if name in ['GridREx', 'CART', 'COSMiK']:
        #    continue
        pred, n, miss = fun(model, scaledTrain, scaledTest, normalization, s, m)
        predicted[name] += list(pred)
        rules[name].append(n)
        missed[name].append(miss)

2491
GridREx
CART
COSMiK
CReEPy
2492
GridREx
CART
COSMiK
CReEPy
2493
GridREx
CART
COSMiK
CReEPy
2494
GridREx
CART
COSMiK
CReEPy
2495
GridREx
CART
COSMiK
CReEPy
2496
GridREx
CART
COSMiK
CReEPy
2497
GridREx
CART
COSMiK
CReEPy
2498
GridREx
CART
COSMiK
CReEPy
2499
GridREx
CART
COSMiK
CReEPy
2500
GridREx
CART
COSMiK
CReEPy
2501
GridREx
CART
COSMiK
CReEPy
2502
GridREx
CART
COSMiK
CReEPy
2503
GridREx
CART
COSMiK
CReEPy
2504
GridREx
CART
COSMiK
CReEPy
2505
GridREx
CART
COSMiK
CReEPy
2506
GridREx
CART
COSMiK
CReEPy
2507
GridREx
CART
COSMiK
CReEPy
2508
GridREx
CART
COSMiK
CReEPy


In [8]:
pd.DataFrame(predicted).to_csv("pred.csv")

In [10]:
pd.DataFrame(rules).to_csv('rules.csv')
pd.DataFrame(missed).to_csv('missed.csv')

In [13]:
p = pd.DataFrame(predicted)

In [22]:
abs(p.COSMiK - p.model).mean()

492505987326.38666

In [23]:
p.describe()

,index,V,model,GridREx,CART,COSMiK,CReEPy
count,11664,11664.000000,11664.000000,1.166400e+04,11664.000000,1.166400e+04,11664.000000
mean,2016-11-01 23:30:00,454.232082,451.424130,9.016708e+09,0.012740,2.897577e+11,455.121051
min,2016-03-04 00:00:00,263.000000,329.304591,-5.506639e+12,-0.890021,-3.547506e+13,284.408661
25%,2016-07-03 11:45:00,368.000000,392.160427,-5.776123e-01,-0.757964,3.301165e+02,395.736008
50%,2016-11-01 23:30:00,431.000000,442.197565,-1.053650e-01,-0.006947,4.555268e+02,448.432503
75%,2017-03-03 11:15:00,525.000000,507.905235,5.485643e-01,0.810958,6.599621e+02,510.909673
max,2017-07-02 23:00:00,761.000000,639.798655,5.516910e+12,1.196391,1.997025e+14,716.249401
std,NaN,107.353337,70.294346,2.476474e+11,0.668172,8.476597e+12,76.059168
